## Download all the necessary files or codes

In [1]:
!git clone https://github.com/kokheng1025/SelfDrivingCar.git

Cloning into 'SelfDrivingCar'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 79 (delta 15), reused 76 (delta 12), pack-reused 0
Unpacking objects: 100% (79/79), done.


In [2]:
cd SelfDrivingCar/CarND-Alexnet-Feature-Extraction/

/content/SelfDrivingCar/CarND-Alexnet-Feature-Extraction


In [3]:
# download training data
!wget https://d17h27t6h515a5.cloudfront.net/topher/2016/October/580a829f_train/train.p

--2018-11-24 12:58:10--  https://d17h27t6h515a5.cloudfront.net/topher/2016/October/580a829f_train/train.p
Resolving d17h27t6h515a5.cloudfront.net (d17h27t6h515a5.cloudfront.net)... 99.84.170.110, 99.84.170.4, 99.84.170.182, ...
Connecting to d17h27t6h515a5.cloudfront.net (d17h27t6h515a5.cloudfront.net)|99.84.170.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120646441 (115M)
Saving to: ‘train.p’

train.p             100%[===================>] 115.06M  81.3MB/s    in 1.4s    

2018-11-24 12:58:11 (81.3 MB/s) - ‘train.p’ saved [120646441/120646441]



In [4]:
# download alexnet weight
!wget https://d17h27t6h515a5.cloudfront.net/topher/2016/October/580d880c_bvlc-alexnet/bvlc-alexnet.npy

--2018-11-24 12:58:13--  https://d17h27t6h515a5.cloudfront.net/topher/2016/October/580d880c_bvlc-alexnet/bvlc-alexnet.npy
Resolving d17h27t6h515a5.cloudfront.net (d17h27t6h515a5.cloudfront.net)... 99.84.170.110, 99.84.170.4, 99.84.170.182, ...
Connecting to d17h27t6h515a5.cloudfront.net (d17h27t6h515a5.cloudfront.net)|99.84.170.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 243861814 (233M)
Saving to: ‘bvlc-alexnet.npy’

bvlc-alexnet.npy    100%[===================>] 232.56M  91.5MB/s    in 2.5s    

2018-11-24 12:58:15 (91.5 MB/s) - ‘bvlc-alexnet.npy’ saved [243861814/243861814]



## TODO:

In [0]:
import pickle
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from alexnet import AlexNet
import pandas as pd
import numpy as np
import time

In [6]:
# TODO: Load traffic sign data.
signNames = pd.read_csv("signnames.csv")
print(signNames.head(5))
classes = 43
epochs = 10
batch_size = 128

   ClassId              SignName
0        0  Speed limit (20km/h)
1        1  Speed limit (30km/h)
2        2  Speed limit (50km/h)
3        3  Speed limit (60km/h)
4        4  Speed limit (70km/h)


In [0]:
# TODO: split data into training and validation sets
training_file = "./train.p"

with open(training_file, mode='rb') as f:
  train = pickle.load(f)

x_train, y_train = train['features'], train['labels']

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.33, random_state=42)

In [0]:
# TODO: Define placeholders and resize operation.
features = tf.placeholder(tf.float32, (None, 32, 32,3))
labels = tf.placeholder(tf.int64, None)
resized = tf.image.resize_images(features, (227, 227))
one_hot_y = tf.one_hot(labels, classes)

In [0]:
# TODO: pass placeholder as first argument to 'AlexNet'
fc7 = AlexNet(resized, feature_extract=True)
fc7 = tf.stop_gradient(fc7)
shape = (fc7.get_shape().as_list()[-1], classes)
fc8W = tf.Variable(tf.truncated_normal(shape, stddev=1e-2))
fc8b = tf.Variable(tf.zeros(classes))
logits = tf.matmul(fc7, fc8W) + fc8b

In [0]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_y)
loss_op = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer()
train_op = optimizer.minimize(loss_operation, var_list=[fc8W, fc8b])
init = tf.global_variables_initializer()

preds = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_op = tf.reduce_mean(tf.cast(preds, tf.float32))

In [0]:
def eval_on_data(X, y, sess):
    total_acc = 0
    total_loss = 0
    for offset in range(0, X.shape[0], batch_size):
        end = offset + batch_size
        X_batch = X[offset:end]
        y_batch = y[offset:end]

        loss, acc = sess.run([loss_op, accuracy_op], feed_dict={features: X_batch, labels: y_batch})
        total_loss += (loss * X_batch.shape[0])
        total_acc += (acc * X_batch.shape[0])

    return total_loss/X.shape[0], total_acc/X.shape[0]

In [26]:
with tf.Session() as sess:
  sess.run(init)
  
  for i in range(epochs):
    # training
    x_train, y_train = shuffle(x_train, y_train)
    t0 = time.time()
    for offset in range(0, x_train.shape[0], batch_size):
      end = offset + batch_size
      sess.run(train_op, feed_dict={features: x_train[offset:end], labels: y_train[offset:end]})
      
    val_loss, val_acc = eval_on_data(x_val, y_val, sess)
    print("EPOCH = ", i+1)
    print("Time: %.3f secconds" %(time.time() - t0))
    print("validation Loss = ", val_loss)
    print("validation accuracy", val_acc)
    print("")

EPOCH =  1
Time: 51.667 secconds
validation Loss =  0.5097713078231032
validation accuracy 0.8690006955945608

EPOCH =  2
Time: 51.453 secconds
validation Loss =  0.3465088032982395
validation accuracy 0.9128217018316717

EPOCH =  3
Time: 51.473 secconds
validation Loss =  0.2697312356006584
validation accuracy 0.9299018471288353

EPOCH =  4
Time: 51.476 secconds
validation Loss =  0.23387273760969243
validation accuracy 0.9393307056186723

EPOCH =  5
Time: 51.469 secconds
validation Loss =  0.2136630137816713
validation accuracy 0.9404127057732437

EPOCH =  6
Time: 51.452 secconds
validation Loss =  0.16897026440055987
validation accuracy 0.9570291367184481

EPOCH =  7
Time: 51.492 secconds
validation Loss =  0.1538236588245428
validation accuracy 0.9612025658860809

EPOCH =  8
Time: 51.546 secconds
validation Loss =  0.14842232677197933
validation accuracy 0.9606615658087951

EPOCH =  9
Time: 51.486 secconds
validation Loss =  0.13220324782194415
validation accuracy 0.965607852229693